In [65]:
##### read input data #####
import pandas as pd 
import os

schedules = {}
for filename in os.listdir('./testdata'):

    # there has been the sign \xa0 in the test data
    with open('./testdata/'+filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    lines = [line.replace('\xa0', '') for line in lines]
    with open('./testdata/'+filename, 'w', encoding='utf-8') as file:
        file.writelines(lines)

    team = filename[:-4]
    raw_df = pd.read_csv('./testdata/'+filename,sep=";").dropna().drop('Schiedsrichter', axis=1)
    raw_df['Team'] = team
    raw_df['Datum'] = pd.to_datetime(raw_df['Datum'], format='%d.%m.%Y %H:%M')
    schedules[team] = raw_df

print(schedules['damen1'])

      Nr.   Tag               Datum                       Heim  \
0     5.0   1.0 2024-09-21 17:00:00         WINGS Leverkusen 2   
2     7.0   2.0 2024-09-28 18:00:00            DJK Frankenberg   
4    15.0   3.0 2024-10-05 18:00:00         SG Bergische Löwen   
6    19.0   4.0 2024-11-02 20:00:00            DJK Frankenberg   
8    30.0   5.0 2024-11-09 16:00:00    VfL AstroStars Bochum 2   
10   31.0   6.0 2024-11-15 20:30:00            DJK Frankenberg   
12   40.0   7.0 2024-11-30 18:30:00    New Basket Oberhausen 2   
14   43.0   8.0 2024-12-07 20:00:00            DJK Frankenberg   
16   50.0   9.0 2024-12-14 16:00:00                  TSV Hagen   
18   55.0  10.0 2024-12-21 16:00:00  Citybasket Recklinghausen   
20   61.0  11.0 2025-01-18 18:00:00            DJK Frankenberg   
22   71.0  12.0 2025-01-25 18:00:00            DJK Frankenberg   
24   73.0  13.0 2025-02-01 16:30:00                UBC Münster   
26   81.0  14.0 2025-02-08 18:00:00            DJK Frankenberg   
28   85.0 

In [66]:
##### filter home schedules from schedules #####
home_schedules = {}
for team, schedule in schedules.items():
    home_schedules[team] = schedule[schedule['Heim'].str.contains('DJK Frankenberg')]

home_schedule = pd.concat([s for s in home_schedules.values()])

print(home_schedules['damen1'])

      Nr.   Tag               Datum             Heim  \
2     7.0   2.0 2024-09-28 18:00:00  DJK Frankenberg   
6    19.0   4.0 2024-11-02 20:00:00  DJK Frankenberg   
10   31.0   6.0 2024-11-15 20:30:00  DJK Frankenberg   
14   43.0   8.0 2024-12-07 20:00:00  DJK Frankenberg   
20   61.0  11.0 2025-01-18 18:00:00  DJK Frankenberg   
22   71.0  12.0 2025-01-25 18:00:00  DJK Frankenberg   
26   81.0  14.0 2025-02-08 18:00:00  DJK Frankenberg   
30   96.0  16.0 2025-02-22 18:00:00  DJK Frankenberg   
34  106.0  18.0 2025-03-15 20:00:00  DJK Frankenberg   
38  116.0  20.0 2025-03-29 18:00:00  DJK Frankenberg   
40  121.0  21.0 2025-04-05 18:00:00  DJK Frankenberg   

                         Gast                   Spielhalle    Team  
2                 UBC Münster  Einhard-Gymnasium (Halle 1)  damen1  
6                  Hürther BC  Einhard-Gymnasium (Halle 1)  damen1  
10        DTV Basketball Köln  Einhard-Gymnasium (Halle 1)  damen1  
14     Talents BonnRhöndorf 2  Einhard-Gymnasium (H

In [67]:
# helping lists and dictionaries
teams = schedules.keys()
games = []
for team in teams:
    for matchday in home_schedules[team]['Tag'].to_list():
        games.append((team, matchday))

games_per_team = {}
for team in teams:
    games_per_team[team] = len(home_schedules[team])
print(games_per_team)

{'damen1': 11, 'damen2': 9, 'damen3': 7, 'herren1': 11, 'herren2': 8, 'u16w': 7}


In [68]:
from collections import defaultdict

games_per_week = defaultdict(list)
for matchday, date, team in home_schedule[['Tag', 'Datum', 'Team']].values:
    calendar_week = date.strftime('%U')
    games_per_week[calendar_week].append((team, matchday))
games_per_week

defaultdict(list,
            {'38': [('damen1', 2.0),
              ('damen2', 2.0),
              ('damen3', 9.0),
              ('herren1', 2.0)],
             '43': [('damen1', 4.0),
              ('damen2', 4.0),
              ('damen3', 2.0),
              ('herren1', 4.0),
              ('herren2', 5.0)],
             '45': [('damen1', 6.0),
              ('damen2', 6.0),
              ('damen3', 4.0),
              ('herren1', 6.0)],
             '48': [('damen1', 8.0),
              ('damen2', 8.0),
              ('damen3', 6.0),
              ('herren1', 8.0),
              ('herren2', 9.0)],
             '02': [('damen1', 11.0), ('herren1', 11.0), ('herren2', 12.0)],
             '03': [('damen1', 12.0), ('damen2', 12.0), ('herren1', 12.0)],
             '05': [('damen1', 14.0),
              ('damen2', 14.0),
              ('damen3', 10.0),
              ('herren1', 14.0),
              ('herren2', 15.0)],
             '07': [('damen1', 16.0),
              ('damen2', 16.0)

In [69]:
from datetime import timedelta

preferred_games_per_team = defaultdict(list)
game_dates_per_team = {}
assignable_dates_per_team = {}
for team in teams:
    for game_date in home_schedules[team]['Datum'].to_list():
        for assignable_date, matchday, playing_team in home_schedule[home_schedule['Team'] != team][['Datum', 'Tag', 'Team']].values:
            if abs(game_date - assignable_date) <= timedelta(hours=3):
                preferred_games_per_team[team].append((playing_team, matchday))

preferred_games_per_team['herren1']

[('damen1', 2.0),
 ('damen2', 2.0),
 ('damen1', 4.0),
 ('damen2', 4.0),
 ('damen2', 6.0),
 ('damen1', 8.0),
 ('damen2', 8.0),
 ('damen1', 11.0),
 ('damen1', 12.0),
 ('damen2', 12.0),
 ('damen1', 14.0),
 ('damen2', 14.0),
 ('damen1', 16.0),
 ('damen2', 16.0),
 ('damen1', 18.0),
 ('damen2', 18.0),
 ('damen1', 20.0),
 ('damen3', 16.0),
 ('damen1', 21.0),
 ('u16w', 11.0)]

In [70]:
##### formulate and solve optimization problem #####
import pulp

teamAssignedToGame = {}

problem = pulp.LpProblem('kampfgerichtsplanung', pulp.LpMaximize)

# define variables
for game in games:
    for team in teams:
        teamAssignedToGame[(game, team)] = pulp.LpVariable(team+'_assigned_to_game_'+str(int(game[1]))+'_of_'+game[0], cat='Binary')
    
# objective function
problem += pulp.lpSum([teamAssignedToGame[(game, team)] for team in teams for game in preferred_games_per_team[team]])

# constraint: exactly one team is assigned to one game
for game in games:
    problem += pulp.lpSum([teamAssignedToGame[(game, team)] for team in teams]) == 1

# constraint: no team is assigned to its own game
for game in games:
    problem += teamAssignedToGame[(game, game[0])] == 0

# constraint: each team is assigned as many times as it has home games
for team in teams:
    problem += pulp.lpSum([teamAssignedToGame[(game, team)] for game in games]) <= games_per_team[team]

# constraint: each team is assigned to at most one game per week
for team in teams:
    for week in games_per_week.keys():
        problem += pulp.lpSum([teamAssignedToGame[(game, team)] for game in games_per_week[week]]) <= 1

problem.solve()

if pulp.LpStatus[problem.status] == 'Optimal':
    for game in games:
        for team in teams:
            if teamAssignedToGame[(game, team)].varValue == True:
                print(f'Team {team} is assigned to game {game}')


Team damen3 is assigned to game ('damen1', 2.0)
Team herren1 is assigned to game ('damen1', 4.0)
Team u16w is assigned to game ('damen1', 6.0)
Team herren1 is assigned to game ('damen1', 8.0)
Team herren1 is assigned to game ('damen1', 11.0)
Team herren1 is assigned to game ('damen1', 12.0)
Team damen3 is assigned to game ('damen1', 14.0)
Team damen3 is assigned to game ('damen1', 16.0)
Team herren1 is assigned to game ('damen1', 18.0)
Team damen2 is assigned to game ('damen1', 20.0)
Team herren1 is assigned to game ('damen1', 21.0)
Team herren1 is assigned to game ('damen2', 2.0)
Team damen3 is assigned to game ('damen2', 4.0)
Team herren1 is assigned to game ('damen2', 6.0)
Team damen3 is assigned to game ('damen2', 8.0)
Team u16w is assigned to game ('damen2', 12.0)
Team herren1 is assigned to game ('damen2', 14.0)
Team herren1 is assigned to game ('damen2', 16.0)
Team damen1 is assigned to game ('damen2', 18.0)
Team damen1 is assigned to game ('damen2', 20.0)
Team damen1 is assigne

In [71]:
##### export solution #####
resulting_schedule = pd.concat([s for s in home_schedules.values()])

assigned_teams = []
for team, matchday in resulting_schedule[['Team', 'Tag']].values:
    for assigned_team in teams:
        if teamAssignedToGame[((team, matchday), assigned_team)].varValue == True:
            assigned_teams.append(assigned_team)
            break

resulting_schedule['Kampfgericht'] = assigned_teams

resulting_schedule.sort_values(by='Datum', inplace=True)

resulting_schedule.to_csv('Kampfgerichtsplan.csv')
resulting_schedule
        

,Nr.,Tag,Datum,Heim,Gast,Spielhalle,Team,Kampfgericht
2,8.0,2.0,2024-09-20 19:00:00,DJK Frankenberg 2,Brander TV Aachen 3,Geschwister-Scholl-Gymnasium (obere Halle),herren2,damen1
1,7.0,2.0,2024-09-28 14:00:00,DJK Frankenberg 2,TV Grafenberg,Einhard-Gymnasium (Halle 1),damen2,herren1
1,7.0,2.0,2024-09-28 16:00:00,DJK Frankenberg,BG Aachen 2,Einhard-Gymnasium (Halle 1),herren1,damen2
2,7.0,2.0,2024-09-28 18:00:00,DJK Frankenberg,UBC Münster,Einhard-Gymnasium (Halle 1),damen1,damen3
0,41.0,9.0,2024-09-28 20:00:00,DJK Frankenberg 3,BG Aachen 2,Einhard-Gymnasium (Halle 1),damen3,damen1
6,18.0,4.0,2024-10-04 19:00:00,DJK Frankenberg 2,Aachener Interkultureller FSV,Geschwister-Scholl-Gymnasium (obere Halle),herren2,u16w
1,10.0,3.0,2024-10-05 13:30:00,DJK Frankenberg Aachen,RheinStars Köln 3,Einhard-Gymnasium (Halle 1),u16w,herren2
8,23.0,5.0,2024-10-28 20:30:00,DJK Frankenberg 2,SG Herzogenrath/Baesweiler 3,Geschwister-Scholl-Gymnasium (obere Halle),herren2,u16w
4,6.0,2.0,2024-11-02 14:00:00,DJK Frankenberg 3,TS Frechen,Einhard-Gymnasium (Halle 1),damen3,damen2
5,19.0,4.0,2024-11-02 16:00:00,DJK Frankenberg 2,BG Shots Kaarst-Büttgen,Einhard-Gymnasium (Halle 1),damen2,damen3


In [72]:
##### show solution #####
import plotly.express as px

assignment_counts = resulting_schedule['Kampfgericht'].value_counts()
fig = px.pie(assignment_counts, names=assignment_counts.index, values=assignment_counts.values, title='Verteilung Kampfgericht')
fig.show()